<a href="https://colab.research.google.com/github/josh1593/clrs-solutions/blob/master/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
os.chdir('drive/MyDrive')

In [7]:
os.chdir('cse_571_grasp')

In [8]:
import pickle

with open('pc_data', 'rb') as f:
    pc_data, grasps_data, labels_data = pickle.load(f)

with open('./test_pc_data','rb') as g:
    test_pc_data, test_grasp_data, test_labels_data = pickle.load(g)


In [9]:
import numpy as np
def normalize_point_cloud(points, fixed_size=1024):
    if len(points) < fixed_size:
        # Pad with zeros if less than fixed_size
        padding = np.zeros((fixed_size - len(points), 3))
        points = np.vstack((points, padding))
    elif len(points) > fixed_size:
        # Randomly sample if more than fixed_size
        indices = np.random.choice(len(points), fixed_size, replace=False)
        points = points[indices]
    return points

def normalize_pc_data(pc_data, fixed_size=1024):
    normalized_pc_data = [normalize_point_cloud(points, fixed_size) for points in pc_data]
    return normalized_pc_data

fixed_size = 1024
normalized_pc_data = normalize_pc_data(pc_data, fixed_size)
normalized_test_data = normalize_pc_data(test_pc_data, fixed_size)


In [10]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class GraspDataset(Dataset):
    def __init__(self, point_clouds, grasps, labels, region_size=0.1, max_points=1024):
        self.point_clouds = point_clouds
        self.grasps = grasps
        self.labels = labels
        self.region_size = region_size
        self.max_points = max_points

    def __len__(self):
        return len(self.point_clouds)

    def transform_to_local(self, point_cloud, grasp):
        # Assuming grasp is a 4x4 transformation matrix
        transformation_matrix = np.linalg.inv(grasp)
        homogenous_coords = np.hstack((point_cloud, np.ones((point_cloud.shape[0], 1))))
        transformed_coords = homogenous_coords @ transformation_matrix.T
        return transformed_coords[:, :3]

    def normalize(self, point_cloud):
        centroid = np.mean(point_cloud, axis=0)
        point_cloud -= centroid
        max_dist = np.max(np.sqrt(np.sum(point_cloud ** 2, axis=1)))
        point_cloud /= max_dist
        return point_cloud

    def __getitem__(self, idx):
      point_cloud = self.point_clouds[idx]
      grasp = self.grasps[idx]
      label = self.labels[idx]

      # Transform point cloud to gripper's local coordinate system
      transformed_point_cloud = self.transform_to_local(point_cloud, grasp)

      # Normalize the point cloud
      normalized_point_cloud = self.normalize(transformed_point_cloud)

      # Ensure the point cloud has 3 channels
      normalized_point_cloud = normalized_point_cloud[:, :3]  # Keep only the XYZ coordinates

      return torch.from_numpy(normalized_point_cloud), torch.tensor(label)




def collate_fn(batch):
    max_points = max(item[0].shape[0] for item in batch)
    max_points = min(max_points, 1024)  # Ensure we don't exceed the max_points limit

    point_clouds = []
    labels = []

    for point_cloud, label in batch:
        if point_cloud.shape[0] > max_points:
            point_cloud = point_cloud[:max_points, :]
        else:
            padding = max_points - point_cloud.shape[0]
            point_cloud = torch.cat([point_cloud, torch.zeros(padding, 3)], dim=0)

        # Ensure the input tensor has 3 channels
        point_cloud = point_cloud.permute(1, 0).unsqueeze(0)  # Transpose and add batch dimension
        point_clouds.append(point_cloud)
        labels.append(label)

    point_clouds = torch.cat(point_clouds, dim=0)
    labels = torch.tensor(labels, dtype=torch.float32)  # Convert labels to tensor and add a singleton dimension

    return point_clouds, labels


In [13]:
dataset = GraspDataset(normalized_pc_data, grasps_data, labels_data)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, collate_fn=collate_fn)

test_dataset = GraspDataset(normalized_test_data, test_grasp_data, test_labels_data)
test_dataloader = DataLoader(test_dataset, batch_size = 128, shuffle = True, collate_fn = collate_fn)


In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader

class PointNet(nn.Module):
    def __init__(self):
        super(PointNet, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)  # Output a single value for binary classification

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  # Increase dropout to 50%

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.mp1 = nn.AdaptiveMaxPool1d(1)  # Global max pooling layer

    def forward(self, x):
        x = x.transpose(2, 1)  # Transpose input tensor to correct shape
        x = self.relu(self.bn1(self.conv1(x.float())))  # Convert input tensor to float data type
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.mp1(x)  # Apply global max pooling
        x = x.view(-1, 1024)  # Flatten the tensor
        x = self.dropout(self.relu(self.bn4(self.fc1(x))))  # Apply dropout
        x = self.dropout(self.relu(self.bn5(self.fc2(x))))  # Apply dropout
        x = self.fc3(x)
        return x

def train_and_evaluate(net, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100, patience=10):
    best_loss = float('inf')
    early_stop_counter = 0
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        net.train()
        running_loss = 0.0
        num_batches = 0

        for i, (inputs, labels) in enumerate(train_loader, 0):
            inputs = inputs.transpose(2, 1).to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs).squeeze()

            if torch.isnan(outputs).any():
                print("NaN values found in outputs")
                continue

            loss = criterion(outputs, labels)

            if torch.isnan(loss).any():
                print("NaN values found in loss")
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1.0)  # Clip gradients
            optimizer.step()

            running_loss += loss.item()
            num_batches += 1

        # Calculate the average training loss for this epoch
        train_losses.append(running_loss / num_batches)

        # Validation phase
        net.eval()
        val_running_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.transpose(2, 1).to(device)
                labels = labels.to(device)
                outputs = net(inputs).squeeze()
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()

                predicted = (outputs > 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss = val_running_loss / len(val_loader)
        val_losses.append(val_loss)
        accuracy = 100 * correct / total

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.2f}%")

        # Step the scheduler with the validation loss
        scheduler.step(val_loss)

    print("Finished Training")
    return train_losses, val_losses

learning_rate = 0.0001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = PointNet().to(device)
optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-4)  # Add weight decay
criterion = nn.BCEWithLogitsLoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)


train_losses, val_losses = train_and_evaluate(net, dataloader, test_dataloader, criterion, optimizer, scheduler, num_epochs=100)

# Plotting the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss over Epochs')
plt.show()

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
<ipython-input-10-9c4dce6a0018>:45: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return torch.from_numpy(normalized_point_cloud), torch.tensor(label)


Epoch [1/100], Train Loss: 0.6180, Val Loss: 0.5889, Val Accuracy: 61.95%
Epoch [2/100], Train Loss: 0.5907, Val Loss: 0.5729, Val Accuracy: 64.20%
Epoch [3/100], Train Loss: 0.5776, Val Loss: 0.5718, Val Accuracy: 63.75%
Epoch [4/100], Train Loss: 0.5679, Val Loss: 0.5748, Val Accuracy: 64.54%
Epoch [5/100], Train Loss: 0.5614, Val Loss: 0.5544, Val Accuracy: 65.75%
Epoch [6/100], Train Loss: 0.5544, Val Loss: 0.5580, Val Accuracy: 64.21%
Epoch [7/100], Train Loss: 0.5496, Val Loss: 0.5509, Val Accuracy: 63.12%
Epoch [8/100], Train Loss: 0.5465, Val Loss: 0.5372, Val Accuracy: 65.97%
Epoch [9/100], Train Loss: 0.5436, Val Loss: 0.5443, Val Accuracy: 67.83%
